In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.set_option('display.max_rows',None)

In [2]:
!ls

 ashrae-energy-prediction.zip   EDA.ipynb	        train.csv
'Basic model.ipynb'	        sample_submission.csv   weather_test.csv
 building_metadata.csv	        test.csv	        weather_train.csv


### Reading data

In [3]:
train_df = pd.read_csv('train.csv')
print(train_df.shape)
train_df.head()

(20216100, 4)


,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [4]:
train_df.isna().sum()

building_id      0
meter            0
timestamp        0
meter_reading    0
dtype: int64

In [5]:
weather_train_df = pd.read_csv('weather_train.csv')
print(weather_train_df.shape)
weather_train_df.head()

(139773, 9)


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [6]:
weather_train_df.isna().sum()

site_id                   0
timestamp                 0
air_temperature          55
cloud_coverage        69173
dew_temperature         113
precip_depth_1_hr     50289
sea_level_pressure    10618
wind_direction         6268
wind_speed              304
dtype: int64

In [7]:
weather_train_df.interpolate(inplace=True)

In [8]:
weather_train_df.isna().sum()

site_id               0
timestamp             0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     1
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [9]:
weather_train_df.fillna(-1.0,inplace = True)

In [10]:
weather_train_df.isna().sum()

site_id               0
timestamp             0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [11]:
weather_train_df.site_id.nunique()

16

**Building_meta data**

In [12]:
building_meatdata_Df = pd.read_csv('building_metadata.csv')
print(building_meatdata_Df.shape)
building_meatdata_Df.head()

(1449, 6)


,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [13]:
building_meatdata_Df.site_id.nunique()

16

In [14]:
building_meatdata_Df.isna().sum()

site_id           0
building_id       0
primary_use       0
square_feet       0
year_built      774
floor_count    1094
dtype: int64

**Change year built and floor count as cat values**

In [15]:
building_meatdata_Df['year_built'] = building_meatdata_Df['year_built'].astype('category')
building_meatdata_Df['year_built'] = building_meatdata_Df['year_built'].cat.codes

building_meatdata_Df['floor_count'] = building_meatdata_Df['floor_count'].astype('category')
building_meatdata_Df['floor_count'] = building_meatdata_Df['floor_count'].cat.codes

In [16]:
building_meatdata_Df.isna().sum()

site_id        0
building_id    0
primary_use    0
square_feet    0
year_built     0
floor_count    0
dtype: int64

In [17]:
building_meatdata_Df.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,106,-1
1,0,1,Education,2720,102,-1
2,0,2,Education,5376,89,-1
3,0,3,Education,23685,100,-1
4,0,4,Education,116607,73,-1


**merge building metadata over weather_train_df**

In [18]:
weather_train_df = pd.merge(weather_train_df,building_meatdata_Df,on='site_id',how='left')
print(weather_train_df.shape)
weather_train_df.head()

(12676166, 14)


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,building_id,primary_use,square_feet,year_built,floor_count
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,0,Education,7432,106,-1
1,0,2016-01-01 00:00:00,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,1,Education,2720,102,-1
2,0,2016-01-01 00:00:00,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,2,Education,5376,89,-1
3,0,2016-01-01 00:00:00,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,3,Education,23685,100,-1
4,0,2016-01-01 00:00:00,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,4,Education,116607,73,-1


In [19]:
weather_train_df.isna().sum()

site_id               0
timestamp             0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
building_id           0
primary_use           0
square_feet           0
year_built            0
floor_count           0
dtype: int64

**merge weather data over train_df**

In [20]:
train_df = pd.merge(train_df,weather_train_df,on=['building_id','timestamp'],how= 'left')
print(train_df.shape)
train_df.head()

(20216100, 16)


,building_id,meter,timestamp,meter_reading,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,primary_use,square_feet,year_built,floor_count
0,0,0,2016-01-01 00:00:00,0.0,0.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,Education,7432.0,106.0,-1.0
1,1,0,2016-01-01 00:00:00,0.0,0.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,Education,2720.0,102.0,-1.0
2,2,0,2016-01-01 00:00:00,0.0,0.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,Education,5376.0,89.0,-1.0
3,3,0,2016-01-01 00:00:00,0.0,0.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,Education,23685.0,100.0,-1.0
4,4,0,2016-01-01 00:00:00,0.0,0.0,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,Education,116607.0,73.0,-1.0


In [21]:
train_df.isna().sum()

building_id               0
meter                     0
timestamp                 0
meter_reading             0
site_id               90495
air_temperature       90495
cloud_coverage        90495
dew_temperature       90495
precip_depth_1_hr     90495
sea_level_pressure    90495
wind_direction        90495
wind_speed            90495
primary_use           90495
square_feet           90495
year_built            90495
floor_count           90495
dtype: int64

In [25]:
train_df[train_df['air_temperature'].isna()].head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
722,565,0,2016-01-01 00:00:00,7.250,4,Education,15326,52,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
723,566,0,2016-01-01 00:00:00,0.519,4,Education,2010,55,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
724,569,0,2016-01-01 00:00:00,246.000,4,Education,86091,62,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
725,570,0,2016-01-01 00:00:00,79.488,4,Education,193202,62,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
726,571,0,2016-01-01 00:00:00,17.000,4,Education,47954,78,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
train_df.building_id.nunique()

1449

### Sample Submission

In [5]:
sub = pd.read_csv('sample_submission.csv')
print(sub.shape)
sub.head()

(41697600, 2)


,row_id,meter_reading
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
